In [1]:
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random, os, pathlib
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils import data

## F-MNIST dataset

In [2]:
import mylibrary.datasets as datasets
import mylibrary.nnlib as tnn
from classes import *

In [3]:
mnist = datasets.FashionMNIST()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

# train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [4]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [5]:
def shuffle_data():
    global train_data, train_label
    randidx = random.sample(range(len(train_label)), k=len(train_label))
    train_data = train_data[randidx]
    train_label = train_label[randidx]

In [6]:
input_size = 784
output_size = 10

learning_rate = 0.0001
batch_size = 50

In [7]:
network_seeds = [147, 258, 369]
# network_seeds = [369]
network_seed = 369

EPOCHS = 20

actf = nn.LeakyReLU
# actf = nn.ELU

learning_rate = 0.005
lambda_ = 2
criterion = nn.BCELoss()
sigmoid = nn.Sigmoid()

use_mixup = False

In [8]:
class MNIST_OneClass_Balanced(data.Dataset):
    
    def __init__(self, data, label, class_index):
        self.data = data
        self.label = label
        self.class_index = class_index
        
        mask = (label==class_index)
        self.label = mask.type(torch.float32).reshape(-1,1)
        self.class_data = torch.nonzero(mask).reshape(-1)
        self.other_data = torch.nonzero(~mask).reshape(-1)
        
        random.seed(network_seed)
        self._shuffle_data_()
        self.count = 0
        
    def __len__(self):
        return 2*len(self.class_data)
    
    def _shuffle_data_(self):
#         randidx = np.random.permutation(len(self.other_data))
        randidx = random.sample(range(len(self.other_data)), k=len(self.other_data))
        self.other_data = self.other_data[randidx]
    
    def __getitem__(self, idx):
        if idx < len(self.class_data):
            idx = self.class_data[idx]
            img, lbl = self.data[idx], self.label[idx]
        else:
            idx = self.other_data[idx-len(self.class_data)]
            img, lbl = self.data[idx], self.label[idx]
            self.count += 1
            if self.count >= len(self.class_data): 
                self._shuffle_data_()
                self.count = 0
        return img, lbl

In [9]:
# class_idx = 0
# train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
# test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

In [10]:
# len(train_dataset), len(test_dataset)

In [11]:
# train_loader_all = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
# test_loader_all = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [12]:
# %matplotlib inline
# # img, lbl = train_dataset[11010]
# img, lbl = test_dataset[10]
# print(lbl)
# plt.imshow(img.reshape(28,28))

## Convex

In [13]:
class UnivariateCNN(nn.Module):
    
    def __init__(self, channels:list, actf=nn.LeakyReLU):
        super().__init__()
        assert len(channels)>1

        layers = []
        for i in range(len(channels)-1):
            la = nn.Conv2d(channels[i], channels[i+1], kernel_size=(5,5), stride=2, padding=1)
            layers.append(la)
            layers.append(actf())
        layers.append(nn.AdaptiveAvgPool2d(1))
        self.features = nn.Sequential(*layers)
        self.fc = nn.Sequential(nn.Linear(channels[-1], 1))
        
    def forward(self, x):
        x = x.reshape(-1,1, 28, 28)
        x = self.features(x)
        s = x.shape
        return self.fc(x.reshape(s[0], s[1]))    
    

class ConvexCNN(nn.Module):
    
    def __init__(self, channels:list, actf=nn.LeakyReLU):
        super().__init__()
        assert len(channels)>1

        layers = []
        for i in range(len(channels)-1):
            la = nn.Conv2d(channels[i], channels[i+1], kernel_size=(5,5), stride=2, padding=1)
            layers.append(la)
            if i>0:
                layers[-1].weight.data *= 0.1
            layers.append(actf())
        layers.append(nn.AdaptiveAvgPool2d(1))
        self.features = nn.Sequential(*layers)
        self.fc = nn.Sequential(nn.Linear(channels[-1], 1))
        
    def forward(self, x):
        x = x.reshape(-1,1,28, 28)
        for i in range(2, len(self.features)-1, 2):
            self.features[i].weight.data.abs_()
        for i in range(0, len(self.fc), 2):
            self.fc[i].weight.data.abs_()
            
        x = self.features(x)
        s = x.shape
        return self.fc(x.reshape(s[0], s[1]))    
    


In [14]:
# cnn = UnivariateCNN([1, 16, 32, 64])
# cnn = ConvexCNN([1, 32, 32])
# cnn(torch.randn(2,1,28,28))

In [15]:
# a = nn.Sequential(nn.Conv2d(1, 1, kernel_size=5, padding=2, stride=2),
#              nn.Conv2d(1,1, kernel_size=5, padding=2, stride=2))
# # b = nn.Conv2d(1, 1, kernel_size=5, padding=2, stride=4)
# b = nn.Conv2d(1, 1, kernel_size=5, padding=2, stride=2, dilation=2)
# x = torch.randn(2,1,32,32)

# a(x).shape, b(x).shape

In [16]:
# a = nn.Sequential(nn.Conv2d(1, 1, kernel_size=5, padding=2, stride=2),
#                  nn.Conv2d(1,1, kernel_size=5, padding=2, stride=2),
#                  nn.Conv2d(1,1, kernel_size=5, padding=2, stride=2))
# b = nn.Conv2d(1, 1, kernel_size=5, padding=2, stride=4, dilation=4)
# x = torch.randn(2,1,28,28)
# # nn.Conv2d()

# a(x).shape, b(x).shape

In [17]:
stat_per_class = []
net_list = []
for class_idx in range(10):
    print(class_idx)
    train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
    test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

    torch.manual_seed(network_seed)
    Net = ConvexCNN([1, 32, 32], actf)
    optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)
    losses = []
    train_accs = []
    test_accs = []

    index = 0
#     EPOCHS = 5
    for epoch in range(EPOCHS):
        train_acc = 0
        train_count = 0
        for xx, yy in train_loader:
            index += 1
            if use_mixup:
                rand_indx = np.random.permutation(len(xx))
                rand_lambda = 1-torch.rand(len(xx), 1)*0.1
                x_mix = rand_lambda*xx+(1-rand_lambda)*xx[rand_indx]
                y_mix = rand_lambda*yy+(1-rand_lambda)*yy[rand_indx]
            else:
                x_mix = xx
                y_mix = yy

            yout = sigmoid(Net(x_mix))    
            loss = criterion(yout, y_mix)
            losses.append(float(loss))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
            train_acc += correct
            train_count += len(outputs)

#             if index%200 == 0:
        train_accs.append(float(train_acc)/train_count*100)
        train_acc = 0
        train_count = 0

        print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
        test_count = 0
        test_acc = 0
        for xx, yy in test_loader:
            with torch.no_grad():
                yout = sigmoid(Net(xx))
            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
            test_acc += correct
            test_count += len(xx)
        test_accs.append(float(test_acc)/test_count*100)
        print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
        print()
                
    ### after each class index is finished training
    stat_per_class.append(
    f'Class: {class_idx} -> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}'
    )
    print(stat_per_class[-1], '\n')
    net_list.append(Net)

In [18]:
## find the classification error from classification per class network.
acc_test = 0
count_test = 0
with torch.no_grad():
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        yy = test_label[index * batch_size:(index + 1) * batch_size]
        yout = []
        for net in net_list:
            yout.append(sigmoid(net(xx)).reshape(-1))
        yout = torch.stack(yout, dim=1).argmax(dim=1)
        acc = (yout == yy).type(torch.float32).sum()
        count_test += len(xx)
        acc_test += acc
        
for s in stat_per_class:
    print(s)
print(f"Total Accuracy (Argmax) is : {float(acc_test/count_test)}")

In [19]:
# ## find the classification acc for confident examples 
# acc_test = 0
# count_test = 0
# certain_percent = 0
# with torch.no_grad():
#     for index in range(len(test_label) // batch_size):
#         xx = test_data[index * batch_size:(index + 1) * batch_size]
#         yy = test_label[index * batch_size:(index + 1) * batch_size]
#         yout = []
#         for net in net_list:
#             yout.append(sigmoid(net(xx)).reshape(-1))
#         yout_ = torch.stack(yout, dim=1)
# #         print(yout_)
# #         print(yy)
# #         break
#         yout_, yout = yout_.max(dim=1)
#         mask = yout_<0.5
#         certain_percent += (mask).type(torch.float32).sum()
#         acc = (yout == yy).type(torch.float32)[~mask].sum()
#         count_test += len(xx)
#         acc_test += acc
        
# print(f"UNConfident {float(certain_percent/count_test)}% (Argmax) is : {float(acc_test/(count_test-certain_percent))}")

## NN

In [20]:
stat_per_class = []
net_list = []
for class_idx in range(10):
    print(class_idx)
    train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
    test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

    torch.manual_seed(network_seed)
    Net = UnivariateCNN([1, 32, 32], actf)
    optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)
    losses = []
    train_accs = []
    test_accs = []

    index = 0
    for epoch in range(EPOCHS):
        train_acc = 0
        train_count = 0
        for xx, yy in train_loader:
            index += 1

            if use_mixup:
                rand_indx = np.random.permutation(len(xx))
                rand_lambda = 1-torch.rand(len(xx), 1)*0.1
                x_mix = rand_lambda*xx+(1-rand_lambda)*xx[rand_indx]
                y_mix = rand_lambda*yy+(1-rand_lambda)*yy[rand_indx]
            else:
                x_mix = xx
                y_mix = yy

            yout = sigmoid(Net(x_mix))    
            loss = criterion(yout, y_mix)
            losses.append(float(loss))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
            train_acc += correct
            train_count += len(outputs)

#             if index%200 == 0:
        train_accs.append(float(train_acc)/train_count*100)
        train_acc = 0
        train_count = 0

        print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
        test_count = 0
        test_acc = 0
        for xx, yy in test_loader:
            with torch.no_grad():
                yout = sigmoid(Net(xx))
            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
            test_acc += correct
            test_count += len(xx)
        test_accs.append(float(test_acc)/test_count*100)
        print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
        print()
                
    ### after each class index is finished training
    stat_per_class.append(
    f'Class: {class_idx} -> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}'
    )
    print(stat_per_class[-1], '\n')
    net_list.append(Net)

In [21]:
## find the classification error from classification per class network.
acc_test = 0
count_test = 0
with torch.no_grad():
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        yy = test_label[index * batch_size:(index + 1) * batch_size]
        yout = []
        for net in net_list:
            yout.append(sigmoid(net(xx)).reshape(-1))
        yout = torch.stack(yout, dim=1).argmax(dim=1)
        acc = (yout == yy).type(torch.float32).sum()
        count_test += len(xx)
        acc_test += acc
        
for s in stat_per_class:
    print(s)
print(f"Total Accuracy (Argmax) is : {float(acc_test/count_test)}")

In [22]:
# ## find the classification acc for confident examples 
# acc_test = 0
# count_test = 0
# certain_percent = 0
# with torch.no_grad():
#     for index in range(len(test_label) // batch_size):
#         xx = test_data[index * batch_size:(index + 1) * batch_size]
#         yy = test_label[index * batch_size:(index + 1) * batch_size]
#         yout = []
#         for net in net_list:
#             yout.append(sigmoid(net(xx)).reshape(-1))
#         yout_ = torch.stack(yout, dim=1)
# #         print(yout_)
# #         print(yy)
# #         break
#         yout_, yout = yout_.max(dim=1)
#         mask = yout_<0.5
#         certain_percent += (mask).type(torch.float32).sum()
#         acc = (yout == yy).type(torch.float32)[~mask].sum()
#         count_test += len(xx)
#         acc_test += acc
        
# print(f"UNConfident {float(certain_percent/count_test)}% (Argmax) is : {float(acc_test/(count_test-certain_percent))}")

# Invex

In [23]:
use_mixup = False
use_check = False
check_every = 2
check_size = 100

m_,s_ = 1, 0

In [24]:
stat_per_class = []
net_list = []
for class_idx in range(10):
    print(class_idx)
    train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
    test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

    torch.manual_seed(network_seed)
    lips_net = UnivariateCNN([1, 32, 32], actf)
    Net = BasicInvexNet(784, lips_net, lambda_)
    optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)
    losses = []
    train_accs = []
    test_accs = []

    index = 0
#     for epoch in range(5):
    for epoch in range(EPOCHS):
        train_acc = 0
        train_count = 0
        for xx, yy in train_loader:
            index += 1
            
            optimizer.zero_grad()
            if use_check and epoch%check_every == 0:
                rand_inp = torch.rand(check_size, 784)*m_+s_
                Net(rand_inp)
                Net.compute_penalty_and_clipper()
                Net.gp.backward(retain_graph=True)
            
            if use_mixup:
                rand_indx = np.random.permutation(len(xx))
                rand_lambda = 1-torch.rand(len(xx), 1)*0.1
                x_mix = rand_lambda*xx+(1-rand_lambda)*xx[rand_indx]
                y_mix = rand_lambda*yy+(1-rand_lambda)*yy[rand_indx]
            else:
                x_mix = xx
                y_mix = yy

            yout = sigmoid(Net(x_mix))   
            Net.compute_penalty_and_clipper()
            loss = criterion(yout, y_mix) + Net.gp
            losses.append(float(loss))

            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            preds = (yy.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == preds).astype(float).sum()
            train_acc += correct
            train_count += len(outputs)

            loss.backward()
            optimizer.step()

#             if index%200 == 0:
        train_accs.append(float(train_acc)/train_count*100)
        train_acc = 0
        train_count = 0

        min_val, gp = float(Net.cond.min()) , float(Net.gp)
        print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}, MinVal:{min_val}, gp: {gp}')
        test_count = 0
        test_acc = 0
        for xx, yy in test_loader:
#                     with torch.no_grad():
            yout = sigmoid(Net(xx))
            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
            test_acc += correct
            test_count += len(xx)
        test_accs.append(float(test_acc)/test_count*100)
        print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
        print()
                
    ### after each class index is finished training
    stat_per_class.append(
    f'Class: {class_idx} -> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}'
    )
    print(stat_per_class[-1], '\n')
    net_list.append(Net)

0
Epoch: 0:240,  Loss:0.23692530393600464, MinVal:0.33153805136680603, gp: 1.9162254716320604e-07
Train Acc:76.11%, Test Acc:90.30%

Epoch: 1:480,  Loss:0.2016894519329071, MinVal:0.1023336723446846, gp: 0.0006862402078695595
Train Acc:91.73%, Test Acc:93.35%

Epoch: 2:720,  Loss:0.12320880591869354, MinVal:0.3461264967918396, gp: 1.46266700085107e-07
Train Acc:93.03%, Test Acc:93.85%

Epoch: 3:960,  Loss:0.19235289096832275, MinVal:0.20166996121406555, gp: 1.8917211491498165e-05
Train Acc:93.65%, Test Acc:93.80%

Epoch: 4:1200,  Loss:0.1601240038871765, MinVal:0.44365864992141724, gp: 1.6671741764895387e-09
Train Acc:94.06%, Test Acc:93.70%

Epoch: 5:1440,  Loss:0.08797988295555115, MinVal:0.16968077421188354, gp: 6.147215026430786e-05
Train Acc:94.34%, Test Acc:94.70%

Epoch: 6:1680,  Loss:0.20766393840312958, MinVal:0.33422863483428955, gp: 1.0775615777447456e-07
Train Acc:94.58%, Test Acc:94.75%

Epoch: 7:1920,  Loss:0.10020942986011505, MinVal:0.4538019597530365, gp: 9.15941655854

Train Acc:93.08%, Test Acc:92.00%

Epoch: 2:720,  Loss:0.3062463700771332, MinVal:0.5335273146629333, gp: 4.3394669801966046e-11
Train Acc:94.04%, Test Acc:91.95%

Epoch: 3:960,  Loss:0.08795896172523499, MinVal:0.28868794441223145, gp: 6.4488244788663e-07
Train Acc:94.54%, Test Acc:94.00%

Epoch: 4:1200,  Loss:0.0688890889286995, MinVal:0.303994357585907, gp: 3.5728683656088833e-07
Train Acc:95.23%, Test Acc:94.70%

Epoch: 5:1440,  Loss:0.18839824199676514, MinVal:0.3920791745185852, gp: 1.3092843254014497e-08
Train Acc:95.38%, Test Acc:94.65%

Epoch: 6:1680,  Loss:0.2698238492012024, MinVal:0.287703275680542, gp: 8.306432732752e-07
Train Acc:95.66%, Test Acc:95.10%

Epoch: 7:1920,  Loss:0.08608528971672058, MinVal:0.553786039352417, gp: 1.918309955328823e-11
Train Acc:95.93%, Test Acc:95.05%

Epoch: 8:2160,  Loss:0.09584604948759079, MinVal:0.5383269786834717, gp: 3.15392191529984e-11
Train Acc:95.93%, Test Acc:94.75%

Epoch: 9:2400,  Loss:0.054753612726926804, MinVal:0.3529022932052

Train Acc:83.61%, Test Acc:82.85%

Epoch: 4:1200,  Loss:0.20059801638126373, MinVal:0.526780366897583, gp: 5.006182274280846e-11
Train Acc:85.64%, Test Acc:85.45%

Epoch: 5:1440,  Loss:0.17083610594272614, MinVal:0.7516535520553589, gp: 6.0923657882894916e-15
Train Acc:86.79%, Test Acc:86.25%

Epoch: 6:1680,  Loss:0.2014707773923874, MinVal:0.7350465059280396, gp: 2.0161813435145073e-14
Train Acc:87.76%, Test Acc:87.10%

Epoch: 7:1920,  Loss:0.41311588883399963, MinVal:0.48853644728660583, gp: 2.4511534468807383e-10
Train Acc:88.19%, Test Acc:86.95%

Epoch: 8:2160,  Loss:0.23671171069145203, MinVal:0.6887109279632568, gp: 1.503332370698593e-13
Train Acc:88.46%, Test Acc:86.55%

Epoch: 9:2400,  Loss:0.2706350088119507, MinVal:0.4995724856853485, gp: 1.430468660190698e-10
Train Acc:89.03%, Test Acc:86.95%

Epoch: 10:2640,  Loss:0.20371194183826447, MinVal:0.6616938710212708, gp: 2.2147110263336794e-13
Train Acc:89.69%, Test Acc:87.40%

Epoch: 11:2880,  Loss:0.31545037031173706, MinVal:0.

Epoch: 5:1440,  Loss:0.06201915442943573, MinVal:0.7823418974876404, gp: 2.1398210064076757e-15
Train Acc:97.97%, Test Acc:97.90%

Epoch: 6:1680,  Loss:0.03540915623307228, MinVal:0.5673704743385315, gp: 9.505384326868871e-12
Train Acc:97.92%, Test Acc:98.20%

Epoch: 7:1920,  Loss:0.02967522107064724, MinVal:0.7082733511924744, gp: 6.998076434745065e-14
Train Acc:98.08%, Test Acc:98.25%

Epoch: 8:2160,  Loss:0.0744667649269104, MinVal:0.7415023446083069, gp: 1.3575993356839539e-14
Train Acc:98.32%, Test Acc:98.25%

Epoch: 9:2400,  Loss:0.06437697261571884, MinVal:0.663670003414154, gp: 2.2499059906810892e-13
Train Acc:98.26%, Test Acc:97.40%

Epoch: 10:2640,  Loss:0.0743129551410675, MinVal:0.688618540763855, gp: 7.439707893795375e-14
Train Acc:98.28%, Test Acc:98.20%

Epoch: 11:2880,  Loss:0.1895832121372223, MinVal:0.8117418885231018, gp: 7.78856641758341e-16
Train Acc:98.53%, Test Acc:98.30%

Epoch: 12:3120,  Loss:0.041259121149778366, MinVal:0.657058835029602, gp: 4.185571431548213

In [25]:
## find the classification error from classification per class network.
acc_test = 0
count_test = 0
with torch.no_grad():
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        yy = test_label[index * batch_size:(index + 1) * batch_size]
        yout = []
        for net in net_list:
            yout.append(sigmoid(net(xx)).reshape(-1))
        yout = torch.stack(yout, dim=1).argmax(dim=1)
        acc = (yout == yy).type(torch.float32).sum()
        count_test += len(xx)
        acc_test += acc
        
for s in stat_per_class:
    print(s)
print(f"Total Accuracy (Argmax) is : {float(acc_test/count_test)}")

Class: 0 -> Train Acc 96.30833333333332 ; Test Acc 95.25
Class: 1 -> Train Acc 99.45833333333334 ; Test Acc 99.1
Class: 2 -> Train Acc 95.14166666666667 ; Test Acc 93.75
Class: 3 -> Train Acc 97.36666666666667 ; Test Acc 95.6
Class: 4 -> Train Acc 95.30833333333332 ; Test Acc 93.65
Class: 5 -> Train Acc 99.25833333333334 ; Test Acc 99.15
Class: 6 -> Train Acc 92.63333333333334 ; Test Acc 88.0
Class: 7 -> Train Acc 98.97500000000001 ; Test Acc 98.5
Class: 8 -> Train Acc 99.54166666666666 ; Test Acc 98.7
Class: 9 -> Train Acc 98.94166666666668 ; Test Acc 98.6
Total Accuracy (Argmax) is : 0.878000020980835


In [26]:
# ## find the classification acc for confident examples 
# acc_test = 0
# count_test = 0
# certain_percent = 0
# with torch.no_grad():
#     for index in range(len(test_label) // batch_size):
#         xx = test_data[index * batch_size:(index + 1) * batch_size]
#         yy = test_label[index * batch_size:(index + 1) * batch_size]
#         yout = []
#         for net in net_list:
#             yout.append(sigmoid(net(xx)).reshape(-1))
#         yout_ = torch.stack(yout, dim=1)
# #         print(yout_)
# #         print(yy)
# #         break
#         yout_, yout = yout_.max(dim=1)
#         mask = yout_<0.5
#         certain_percent += (mask).type(torch.float32).sum()
#         acc = (yout == yy).type(torch.float32)[~mask].sum()
#         count_test += len(xx)
#         acc_test += acc
        
# print(f"UNConfident {float(certain_percent/count_test)} (Argmax) is : {float(acc_test/(count_test-certain_percent))}")

In [27]:
## Check the constraint on large number of points, including training and test data.

for class_idx in range(10):
    correct = 0
    count = 0
    net = net_list[class_idx]
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)
        
    for index in range(len(train_label) // batch_size):
        xx = train_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)

    for i in tqdm(range(20000)):
        xx = torch.rand(batch_size, 784)
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)

    print(f"Class: {class_idx} -> Correct {correct/count*100:.4f}% on {count} input points")

100%|██████████| 20000/20000 [03:34<00:00, 93.16it/s]


Class: 0 -> Correct 99.9739% on 1070000 input points


100%|██████████| 20000/20000 [03:32<00:00, 93.93it/s]


Class: 1 -> Correct 99.9064% on 1070000 input points


100%|██████████| 20000/20000 [04:04<00:00, 81.96it/s]


...

Class: 7 -> Correct 100.0000% on 1070000 input points


100%|██████████| 20000/20000 [05:12<00:00, 64.08it/s]


Class: 8 -> Correct 99.9998% on 1070000 input points


100%|██████████| 20000/20000 [03:34<00:00, 93.12it/s]

Class: 9 -> Correct 99.9999% on 1070000 input points


In [28]:
## only on training and testing data
for class_idx in range(10):
    correct = 0
    count = 0
    net = net_list[class_idx]
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)
        
    for index in range(len(train_label) // batch_size):
        xx = train_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)

    print(f"Class: {class_idx} -> Correct {correct/count*100:.4f}% on {count} input points")

Class: 0 -> Correct 100.0000% on 70000 input points
Class: 1 -> Correct 100.0000% on 70000 input points
Class: 2 -> Correct 100.0000% on 70000 input points
Class: 3 -> Correct 99.9971% on 70000 input points
Class: 4 -> Correct 99.9929% on 70000 input points
Class: 5 -> Correct 100.0000% on 70000 input points
Class: 6 -> Correct 99.9986% on 70000 input points
Class: 7 -> Correct 100.0000% on 70000 input points
Class: 8 -> Correct 99.9971% on 70000 input points
Class: 9 -> Correct 99.9986% on 70000 input points


In [29]:
# net(xx)

In [30]:
# net.compute_penalty_and_clipper()

In [31]:
# net